In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import scatter
import torch.nn as nn
import torch

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

softmax = nn.Softmax(dim=-1)

for step, data in enumerate(loader):
    edge_index, batch = data.edge_index, data.batch

    x = torch.rand(batch.size(0))
    # print(x)

    num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')
    print(num_nodes)
    batch_size, max_num_nodes = num_nodes.size(0), int(num_nodes.max())

    cum_num_nodes = torch.cat(
        [num_nodes.new_zeros(1),
         num_nodes.cumsum(dim=0)[:-1]], dim=0)

    index = torch.arange(batch.size(0), dtype=torch.long)
    # print(index)
    # index - cum_num_nodes[batch] 得到的是每个点在各自图中的编号
    index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
    # m = batch * max_num_nodes
    # print(index)

    dense_x = x.new_full((batch_size * max_num_nodes, ), -60000.0)
    # print(dense_x.size())
    dense_x[index] = x
    # print(x[0:max_num_nodes*3])
    # print(dense_x[0:max_num_nodes*3])
    dense_x = dense_x.view(batch_size, max_num_nodes)
    # print(dense_x[:3])
    dense_x = softmax(dense_x)
    # print(dense_x[-10:])
    probs, perm = dense_x.sort(dim=-1, descending=True)
    # print(perm)
    cum_probs = torch.cumsum(probs, dim=-1)

    nucleus = (cum_probs < 0.5)+0
    k = torch.count_nonzero(nucleus, dim=1).reshape(-1)
    k = torch.clamp(k, min=1)
    perm = perm + cum_num_nodes.view(-1, 1)
    # print(perm)
    perm = perm.view(-1)

    index = torch.cat([
        torch.arange(k[i], device=x.device) + i * max_num_nodes
        for i in range(batch_size)
    ], dim=0)
    print(index)

    perm = perm[index]

    break

In [ ]:
import torch
import torch.nn as nn

softmax = nn.Softmax(dim=-1)
temperature = 1.0

x = torch.rand(20, 32)
x = x/float(temperature)
probs, indices = x.sort(dim=-1, descending=True)
# print(x)
# print(probs)
sm = softmax(probs)
# print(sm)
grad = sm[:, 1:] - sm[:, :-1]
grad = grad[:, 1:] - grad[:, :-1]
# print(grad)
only_pos = torch.abs(grad)
sum = torch.sum(only_pos, dim=1).view(-1, 1)
sec_weights = only_pos/sum
# print(sec_weights)
cum_weights = (torch.cumsum(sec_weights, dim=1) > 0.9)+0
# print(cum_weights)
tail_ids = torch.argmax(cum_weights, dim=1)+1
num_samples = tail_ids.min()
tail_ids = tail_ids.view(-1, 1)
tail_min_vals = torch.gather(x, dim=1, index=tail_ids)
# print(x[0:2])
# print(tail_min_vals[0:2])
pruned = (x < tail_min_vals)+0
k = torch.count_nonzero(pruned, dim=1).reshape(-1)
print(k)
mm = torch.arange(20)
print(mm)
out = torch.where(k < mm, k, mm)
print(out)
# print(pruned[0:2])
# pruned = torch.where(
#     x < tail_min_vals, torch.ones_like(x, dtype=x.dtype)*1e10, x)
# sampler = torch.multinomial(pruned, num_samples)
# out = sampler + (torch.arange(20)*32).view(-1, 1)
# print(pruned)
# print(sampler)
# print(out)